# Analysis of GPR

#### Author: D. Li
#### 23.3.2020

This is the analytical solution of GPR model in a 0D (point) problem. 
Analytical solutions are based on Maurizio's crack paper and GPRDI documentation.

Two key variables are:

1) shear relaxation time $\tau$

$\tau = tau_0*\exp(\alpha - \beta * Y)$

2) damage coefficient $\xi$

$\xi = \frac{\mu_2 \mu_2}{[\mu_1 -(\mu_1-\mu_2)\frac{t-ti}{t-te}]^2} \frac{1}{ti-te} (ti < t< te)$

Damage function $\xi$ is introduced in the period of [ti, te] in the given format. This damage coefficient will result in a weakening of the materials and thus release shear stress. We assume an initial strain tensor and constant strain rate tensor in the analysis.

We further define an equivalent coefficient as the ratio between YY and P. The shape of this equivalent coefficient approximates linear slip weakening (LSW) as in dynamic simulation. 


In [2]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from functions import *

%matplotlib inline

In [101]:
# initialize parameters and variables in GPR model 0D
# mu1 lambda1 k mu2 lambda2 K1, K2: shear modulus, Lame's and bulk modulus
# ti te: start and ending time
# alpha1, beta1, alpha2, beta2: in the definition of shear relaxation time tau1 and tau2
# strain0 and strainR: initial strain tensor and constant strain rate tensor.
# D: fault zone thickness: in calculating V_{plastic} = 2*strainRII * D

mu1 = 30e9;
lambda1 = 30e9;

k = 0.2;

lambda2 = lambda1 + 2/3*(1-k)*mu1;  # eq 56
# mu2 = lambda1-lambda2+2*k*mu1; % eq 53
mu2 = k*mu1; # eq 54

K1 = lambda1 + 2/3*mu1;
K2 = lambda2 + 2/3*mu2;

ti = 0.2;
te = 0.4;

alpha1 = 36;
beta1 = 0;
alpha2 = 32;
beta2 = 1e-6;
tau01= 1e30;
tau02 = 3;

c0 = 0.0;

t = np.linspace(0,1,1001); # time integral
YY = np.zeros(np.size(t)); # second invariant
c = np.zeros(np.size(t)); # Xi
tauM = np.zeros(np.size(t)); # tauM
P = np.zeros(np.size(t)); # hydrostatic stress
vp = np.zeros(np.size(t)); # v_{plastic}
slp = np.zeros(np.size(t)); # slip (meter)
sxy = np.zeros(np.size(t));
sxx = np.zeros(np.size(t));
syy = np.zeros(np.size(t));

# constant strain rate tensor
strainR_11 = 1e-4;
strainR_22 = 1e-6;

strainR_33 = 0.0e-4;
strainR_12 = 0.0e-4;
strainR_13 = 0.0e-7;
strainR_23 = 0;

strainR = np.matrix([[strainR_11,strainR_12,strainR_13],
           [strainR_12, strainR_22, strainR_23],
          [ strainR_13, strainR_23, strainR_33 ]]); 

# initial strain tensor

strain11 = 1e-5;
strain22 = 1e-5;

strain12 = -0.0e-11;
strain13 = -0.0e-11;
strain33 = 0.0e-11;
strain23 = 0;

strain0 = np.matrix([[strain11,strain12,strain13],
              [strain12,   strain22,   strain23],
              [strain13, strain23 ,strain33]]);

D = 1; # in meter

P.shape

# try different damage function

def GetXi2(c0,t,dt,mu1,mu2,ti,te):
    if (t<ti or t>te):
        dcdt = 0;
    else:
        dcdt = mu2*mu1*(mu1-(mu1-mu2)*(t-ti)/(te-ti))**-2.0*(te-ti)**-1.0;
#         Maurizio
#       dcdt = (t-ti)/(te-ti); # linear 
#        dcdt = (mu2 + (mu1-mu2)/(1+ ((t-ti)/(te-ti))^2 ) ^1/2 )/mu2;

    c = c0+dt*dcdt;
    return c;




In [102]:
# time evolution 

for i in range(1,t.size):
    
    c[i] = GetXi2(c[i-1],t[i],t[i]-t[i-1],mu1,mu2,ti,te);
    
    mu_a =  (1-c[i])*mu2 + c[i] * mu1; #% eq 57
    K_a  =  (1-c[i])*K2 + c[i] * K1;  #% 

    M = mu1*mu2/mu_a;   # % eq. 45
    L = K1*K2/K_a -2/3 *mu1* mu2 /mu_a;
    
    strain = strain0 + (t[i]-t[i-1])*strainR;
    
    vp[i] = CalcVp(strain, strain0, D, t[i]-t[i-1]);   #% Vp = 2*strain_rate(II)*D
    
    slp[i] = slp[i-1] + vp[i]*(t[i]-t[i-1]);

    sigma,P[i] = StrainToStress(strain,L,M);
    sxy[i] = sigma[0,1];
    sxx[i] = sigma[0,0];
    syy[i] = sigma[1,1];
    
    YY[i] = second(sigma);

    tau1 = calTauI(tau01,alpha1,beta1,YY[i]);
    tau2 = calTauI(tau02,alpha2,beta2, YY[i]);

    tauM[i] = ((1-c[i])/tau1/mu1 + c[i]/tau2/mu2)**-1.0; #% eq 49

    phi = 1/2*(sigma - sigma.trace()/3*np.eye(3))/tauM[i] ; #% eq 48
    
    strain_end = strain  +  (strainR - phi)*(t[i]-t[i-1]) ; # % eq 52
    
    strain0 = strain_end;
    


/Users/duoli/anaconda3/envs/obspy/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


In [104]:
# plot variables with slip distance (m)
fig,ax = plt.subplots(ncols=3,nrows=2,figsize=(16,8));

#ax[0,0].set_title('Second invariant stress (Pa)');
ax[0,0].set_xlabel('slip (m)');
ax[0,0].set_ylabel('YY (Pa)');
ax[0,0].plot(slp,YY,'-r');

ax[0,0].plot(np.array([slp[200],slp[200]]),np.array([np.max(YY),np.min(YY)]),'-.k');
ax[0,0].plot(np.array([slp[400],slp[400]]),np.array([np.max(YY),np.min(YY)]),'-.k');


#ax[0,1].set_title('damage coeff.)');
ax[0,1].set_xlabel('slip (m)');
ax[0,1].set_ylabel('Xi (Pa)');
ax[0,1].plot(slp,c,'-k');


#ax[0,2].set_title('plastic slip rate');
ax[0,2].set_xlabel('slip (m)');
ax[0,2].set_ylabel('Vp (m/s)');
ax[0,2].plot(slp[1:],vp[1:],'-c');


#ax[1,0].set_title('hydrostatic stress (Pa)');
ax[1,0].set_xlabel('slip (m)');
ax[1,0].set_ylabel('P (Pa)');
ax[1,0].plot(slp,P,'-b');

#ax[1,1].set_title('equivalent friction coeff (YY/P)');
ax[1,1].set_xlabel('slip (m)');
ax[1,1].set_ylabel(' YY/P');
ax[1,1].plot(slp,YY/P,'-m');

#ax[1,1].set_title('Sxx');
ax[1,2].set_xlabel('slip (m)');
ax[1,2].set_ylabel(' sxx (Pa)');
ax[1,2].plot(slp[1:],sxx[1:],'-m');
